In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
import operator
import csv

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# $example off$
from pyspark.sql import SparkSession
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession\
    .builder\
    .appName("RandomForestClassifierExample")\
    .getOrCreate()

train_datafile = "file:///home/stone/Train-label-28x28.csv"
test_datafile = "file:///home/stone/Test-label-28x28.csv"

train_df = spark.read.csv(train_datafile,inferSchema="true")
test_df = spark.read.csv(test_datafile,inferSchema="true")

In [ ]:
layers = [784, 100, 10]
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=30, seed=1234)

In [ ]:
train_assembler = VectorAssembler(inputCols=train_df.columns[1:],
    outputCol="features")
train_vectors = train_assembler.transform(train_df).select(col(train_df.columns[0]).alias("label"),"features")
train_vectors.show(5)

In [ ]:
test_assembler = VectorAssembler(inputCols = test_df.columns[1:], outputCol="features")
test_vectors = test_assembler.transform(test_df).select(col(train_df.columns[0]).alias("label"), "features")
test_vectors.show(5)

In [ ]:
model = trainer.fit(train_vectors)
test_result = model.transform(test_vectors)
test_result.show(5)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(test_result)
print("Test Error = %g" % (1.0 - accuracy))
print("args:",layers)
print("Accuracy:",accuracy)